<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="##-Predicting-open-value-of-crude-oil-after-60-days-taking-past-data-into-account." data-toc-modified-id="#-Predicting-open-value-of-crude-oil-after-60-days-taking-past-data-into-account.-1"><span class="toc-item-num">1&nbsp;&nbsp;</span># Predicting open value of crude oil after 60 days taking past data into account.</a></span></li></ul></div>

In [1]:
import numpy as np
import pandas as pd
import math
import sklearn
import datetime
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# split data in 80%/10%/10% train/validation/test sets
valid_set_size_percentage = 10 
test_set_size_percentage = 10 


# # Predicting open value of crude oil after 60 days taking past data into account.

In [4]:
df = pd.read_excel(r"D:/Research project/share data/08-08-2008 to 01-07-2016/Crude data.xlsx")

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2403 entries, 0 to 2402
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    2403 non-null   datetime64[ns]
 1   symbol  2403 non-null   object        
 2   open    2403 non-null   float64       
 3   high    2403 non-null   float64       
 4   low     2403 non-null   float64       
 5   close   2403 non-null   float64       
 6   volume  2403 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 131.5+ KB


In [13]:
data = df[["Date","open"]]

In [14]:
data

,Date,open
0,2008-08-08,119.900002
1,2008-08-10,0.000000
2,2008-08-11,115.199997
3,2008-08-12,114.529999
4,2008-08-13,113.089996
...,...,...
2398,2016-06-26,0.000000
2399,2016-06-27,47.810001
2400,2016-06-28,46.590000
2401,2016-06-29,48.060001


In [15]:
data_training = data[data['Date']<'2015-01-01'].copy()
data_test = data[data['Date']>='2015-01-01'].copy()

In [16]:
data_training = data_training.drop(['Date'], axis = 1)


In [17]:
scaler = MinMaxScaler()
data_training = scaler.fit_transform(data_training)
data_training

array([[0.98642533],
       [0.        ],
       [0.94775808],
       ...,
       [0.45290002],
       [0.44195804],
       [0.44319208]])

In [18]:
X_train = []
y_train = []

for i in range(60, data_training.shape[0]):
    X_train.append(data_training[i-60:i])
    y_train.append(data_training[i, 0])
    
X_train, y_train = np.array(X_train), np.array(y_train)

In [19]:
X_train.shape

(1886, 60, 1)

In [20]:
X_train

array([[[0.98642533],
        [0.        ],
        [0.94775808],
        ...,
        [0.64788151],
        [0.6055121 ],
        [0.59481696]],

       [[0.        ],
        [0.94775808],
        [0.94224596],
        ...,
        [0.6055121 ],
        [0.59481696],
        [0.        ]],

       [[0.94775808],
        [0.94224596],
        [0.93039896],
        ...,
        [0.59481696],
        [0.        ],
        [0.59366518]],

       ...,

       [[0.68432742],
        [0.        ],
        [0.68391604],
        ...,
        [0.        ],
        [0.45956396],
        [0.        ]],

       [[0.        ],
        [0.68391604],
        [0.68087207],
        ...,
        [0.45956396],
        [0.        ],
        [0.45290002]],

       [[0.68391604],
        [0.68087207],
        [0.67914439],
        ...,
        [0.        ],
        [0.45290002],
        [0.44195804]]])

In [21]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

In [22]:
regressor = Sequential()

regressor.add(LSTM(units = 60, activation = 'relu', return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 60, activation = 'relu', return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 80, activation = 'relu', return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 120, activation = 'relu'))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

In [23]:
regressor.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 60, 60)            14880     
_________________________________________________________________
dropout (Dropout)            (None, 60, 60)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 60, 60)            29040     
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 60)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 60, 80)            45120     
_________________________________________________________________
dropout_2 (Dropout)          (None, 60, 80)            0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 120)               9

In [24]:
regressor.compile(optimizer='adam', loss = 'mean_squared_error')
regressor.fit(X_train, y_train, epochs=50, batch_size=32)

Train on 1886 samples
Epoch 1/50
1886/1886 [==============================] - 12s 6ms/sample - loss: 0.1308
Epoch 2/50
1886/1886 [==============================] - 7s 4ms/sample - loss: 0.0850
Epoch 3/50
1886/1886 [==============================] - 7s 4ms/sample - loss: 0.0851
Epoch 4/50
1886/1886 [==============================] - 7s 4ms/sample - loss: 0.0825
Epoch 5/50
1886/1886 [==============================] - 7s 4ms/sample - loss: 0.0851
Epoch 6/50
1886/1886 [==============================] - 7s 4ms/sample - loss: 0.0836
Epoch 7/50
1886/1886 [==============================] - 7s 4ms/sample - loss: 0.0834
Epoch 8/50
1886/1886 [==============================] - 7s 4ms/sample - loss: 0.0841
Epoch 9/50
1886/1886 [==============================] - 8s 4ms/sample - loss: 0.0830
Epoch 10/50
1886/1886 [==============================] - 7s 4ms/sample - loss: 0.0826
Epoch 11/50
1886/1886 [==============================] - 8s 4ms/sample - loss: 0.0813
Epoch 12/50
1886/1886 [=================

In [25]:
data_test.head()

,Date,open
1946,2015-01-02,53.759998
1947,2015-01-04,0.000000
1948,2015-01-05,52.610001
1949,2015-01-06,50.000000
1950,2015-01-07,48.000000


In [26]:
data_training1 = data[data['Date']<'2015-01-01'].copy()


In [27]:
past_60_days = data_training1.tail(60)

In [28]:
df = past_60_days.append(data_test, ignore_index = True)
df = df.drop(['Date'], axis = 1)
df.head()

,open
0,82.760002
1,82.550003
2,80.419998
3,81.949997
4,0.000000


In [29]:
inputs = scaler.transform(df)
inputs

array([[0.68087207],
       [0.67914439],
       [0.6616207 ],
       [0.6742081 ],
       [0.        ],
       [0.6686137 ],
       [0.66343066],
       [0.67091733],
       [0.67404359],
       [0.66672151],
       [0.        ],
       [0.66301928],
       [0.64352118],
       [0.6364459 ],
       [0.64886879],
       [0.64055943],
       [0.        ],
       [0.64582475],
       [0.63496502],
       [0.63759768],
       [0.63307277],
       [0.6117647 ],
       [0.        ],
       [0.62468119],
       [0.62106126],
       [0.6106952 ],
       [0.61085971],
       [0.62764294],
       [0.        ],
       [0.63035789],
       [0.62122583],
       [0.60773345],
       [0.        ],
       [0.        ],
       [0.54298641],
       [0.56964209],
       [0.5561497 ],
       [0.55450432],
       [0.54956809],
       [0.        ],
       [0.53854379],
       [0.51838747],
       [0.52102016],
       [0.50283832],
       [0.48679553],
       [0.        ],
       [0.4695187 ],
       [0.455

In [30]:
X_test = []
y_test = []

for i in range(60, inputs.shape[0]):
    X_test.append(inputs[i-60:i])
    y_test.append(inputs[i, 0])

X_test, y_test = np.array(X_test), np.array(y_test)
X_test.shape, y_test.shape

((457, 60, 1), (457,))

In [31]:
y_pred = regressor.predict(X_test)

In [32]:
scaler.scale_

array([0.00822707])

In [33]:
scale = 1/0.00822707
scale

121.54995642434064

In [34]:
y_pred = y_pred*scale
y_test = y_test*scale

In [35]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import r2_score


In [36]:
k = mae(y_pred, y_test)
print('Training Mean Absolute Error', k )

Training Mean Absolute Error 8.407415462451311


In [37]:
sklearn.metrics.r2_score(y_test, y_pred)

0.6532543371575924

In [38]:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(y_test, y_pred)
mse

123.82313009256397

In [39]:
#RMSE
Rmse=np.sqrt(mse)
Rmse

11.127584198403712

In [40]:
# adjusted r2
var_test = y_test.var()
var_test
Adj_rsquare = 1-(mse/var_test)
Adj_rsquare

0.6532543371575924